In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Import LIBS**

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,MaxPool2D,Dropout,Conv2D,InputLayer,MaxPooling2D,UpSampling2D,Conv2DTranspose
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import keras
from keras import layers
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os

# **Import the train and test datasets**

In [ ]:

train_dir=r"/kaggle/input/skin-cancer-malignant-vs-benign/train/"
test_dir=r"/kaggle/input/skin-cancer-malignant-vs-benign/test/"

# **Preprocessing The Data**

In [ ]:
categories = ['benign','malignant']
y_train = []
x_train = []
x_test = []
y_test = []
Batch_Size = 32
INIT_LR = 1e-4
EPOCHES = 10

# **Image Size**

In [ ]:
from PIL import Image
def get_image_size(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
    return width, height

image_path='/kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/728.jpg'
width, height = get_image_size(image_path)
print(f"Image size: {width}x{height}")


# **Reading The Images From The Train Dataset**

In [ ]:
for category in categories:
    path = os.path.join(train_dir,category)
    for img in os.listdir(path):
        img_path = os.path.join(path,img)
        image = cv2.imread(img_path)
        image = cv2.resize(image,(224,224))
        image = image / 255
        x_train.append(image)
        y_train.append(category)

# **Reading The Images From The Test Dataset**

In [ ]:
for category in categories:
    path = os.path.join(test_dir,category)
    for img in os.listdir(path):
        img_path = os.path.join(path,img)
        image = cv2.imread(img_path)
        image = cv2.resize(image,(224,224))
        image = image / 255
        x_test.append(image)
        y_test.append(category)

# **Define the autoencoder architecture**

In [ ]:
encoding_dim = 32

input_image = keras.Input(shape=(224*224*3,))
x= layers.Dense(256, activation='relu')(input_image)
x= layers.Dense(128, activation='relu')(x)

encoded = layers.Dense(encoding_dim, activation='relu') (x)
x= layers.Dense(128, activation='relu')(encoded)
x= layers.Dense(256, activation='relu')(x)

decoded = layers.Dense(224*224*3, activation='sigmoid')(x)
autoencoder = keras.Model(input_image, decoded)

In [ ]:
encoder = keras.Model(input_image, encoded)

In [ ]:
encoded_input = keras.Input(shape=(encoding_dim,))
x= autoencoder.layers[-3](encoded_input)
x= autoencoder.layers[-2](x)
decoder_layer = autoencoder.layers[-1](x)
decoder = keras.Model(encoded_input, decoder_layer)

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
x_train = np.array(x_train,dtype='float32')
x_test = np.array(x_test,dtype='float32')
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)


In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
encoded_imgs

In [ ]:
import matplotlib.pyplot as plt

n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    original_img = x_test[i].reshape(224, 224, 3)  # Adjust the reshape dimensions to match the size of x_test[i]
    plt.imshow(original_img)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    reconstructed_img = decoded_imgs[i].reshape((224, 224, 3))  # Adjust the reshape dimensions to match the size of decoded_imgs[i]
    plt.imshow(reconstructed_img)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()